In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%run -i '/home/ivan/scripts/test/DataScienceTools/DStools.py'
def plotColorScatter_x(DataFrame ,xvalue = '0',yvalue = '1', sizevalue = 'size', outputFilePath='/abc/test.html',plotWidth = 750, plotHeight = 750, readList = ['1','2'],titleName='tSNE', colorColumn="Category", colorPattern=viridis):
    factors = ['CD69-DP','CD69+DP','CD4+CD8low','TCR+DP','CD4SP_immature','CD4SP_mature','CD8SP']
    if len(factors)<8:
        color_map = factor_cmap(colorColumn,factors=factors,palette=['#f03b20','#feb24c','#ffeda0','#636363','#a1d99b','#31a354','#3182bd'])
    else:
        color_map = factor_cmap(colorColumn,factors=factors,palette=colorPattern(len(factors)))
    hover = HoverTool()
    tooltipString = ""
    for ele in readList:
        ele=str(ele)
        readTuple = (ele.lower(),ele)
        tooltipString = tooltipString + """<br><font face="Arial" size="4">%s: @%s<font>""" % readTuple
    hover.tooltips = tooltipString
    tools= [hover,WheelZoomTool(),PanTool(),BoxZoomTool(),ResetTool(),SaveTool()]
    source= ColumnDataSource(DataFrame)
    output_file(outputFilePath)
    p = figure(plot_width = plotWidth, plot_height = plotHeight, tools=tools,title=titleName,toolbar_location='right',x_axis_label=xvalue.lower(),y_axis_label=yvalue.lower(),background_fill_color='white',title_location = 'above')
    p.title.text_font_size='15pt'
    p.title.align = 'center'
    p.xaxis.axis_label_text_font_size='12pt'
    p.yaxis.axis_label_text_font_size='12pt'
    p.x_range = Range1d(DataFrame[xvalue].min()*1.1,DataFrame[xvalue].max()*1.1)
    p.y_range = Range1d(DataFrame[yvalue].min()*1.1,DataFrame[yvalue].max()*1.1)
    p.circle(x = xvalue,y = yvalue,size=sizevalue,source=source,color=color_map,legend=colorColumn)
    p.legend.location = "top_left"
    p.toolbar.active_scroll=p.select_one(WheelZoomTool)
    if outputFilePath.endswith('png'):
        export_png(p, filename=outputFilePath)
    else:
        output_file(outputFilePath)
        show(p)


In [35]:
logcountsFull = pd.read_csv('/home/ivan/Desktop/Project2/MyData/forPython/logNormalWithTCR.csv', sep=',')
logcountsFull.columns=[i.upper() for i in logcountsFull.columns]
full_gene = logcountsFull.columns.tolist()
full_gene.sort()
factors = pd.read_csv('/home/ivan/Desktop/Project2/MyData/forPython/factors_TCR.csv', sep=',')
gene_df_full = pd.merge(logcountsFull[full_gene],factors['group'],left_index=True,right_index=True)
gene_df_full = gene_df_full[gene_df_full['group']!='lowTCR_Rest']
gene_df_full.loc[gene_df_full['group']=='WT1','group']='CD69-DP'
gene_df_full.loc[gene_df_full['group']=='WT2','group']='CD69+DP'
gene_df_full.loc[gene_df_full['group']=='WT3','group']='CD4+CD8low'
gene_df_full.loc[gene_df_full['group']=='WT4','group']='CD4SP'
gene_df_full.loc[gene_df_full['group']=='WT5','group']='CD8SP'
gene_df_full.loc[gene_df_full['group']=='highTCR_Rest','group']='TCR+DP'
de_gene = pd.read_csv('/home/ivan/Desktop/Project2/MyData/forPython/degene.csv', sep=',')
de_gene=de_gene.index.tolist()
de_gene=[i.upper() for i in de_gene]
seurat_degenes=pd.read_csv('/home/ivan/Desktop/Project2/MyData/pipeline/seurat_degene.txt', sep='\t')
seurat_degenes = seurat_degenes['x'].tolist()
seurat_degenes = [gene.replace('.','-') for gene in seurat_degenes]

In [5]:
#calculate the dropout rate
gene_dfc=gene_df_full[gene_df_full['group']!='TCR+DP'].copy()
gene_dfc['cell']=gene_dfc.index
drode_de_gene_df = Drode_DE_gene_detection(gene_dfc,full_gene,feature='gene',id_col='cell',group_col='group',is_unify=True)
drode_de_gene_df.to_csv(
    path_or_buf="/home/ivan/Desktop/Project2/MyData/pipeline/DE_gene_fullWT4.csv", 
    sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, 
    mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', 
    chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, escapechar=None, decimal='.')

0610009B22RIK
0610009O20RIK
0610010F05RIK
0610012G03RIK
0610030E20RIK
0610037L13RIK
1110002L01RIK
1110004E09RIK
1110004F10RIK
1110008F13RIK
1110008L16RIK
1110008P14RIK
1110012L19RIK
1110032A03RIK
1110034G24RIK
1110038F14RIK
1110051M20RIK
1110059E24RIK
1110059G10RIK
1190002N15RIK
1190007I07RIK
1300002E11RIK
1500011B03RIK
1500011K16RIK
1500015A07RIK
1600002K03RIK
1600014C10RIK
1600020E01RIK
1700008J07RIK
1700010I14RIK
1700016K19RIK
1700017B05RIK
1700020I14RIK
1700021F05RIK
1700028J19RIK
1700030K09RIK
1700037H04RIK
1700040D17RIK
1700041G16RIK
1700055D18RIK
1700056N10RIK
1700061G19RIK
1700062C10RIK
1700066M21RIK
1700096K18RIK
1700097N02RIK
1700102P08RIK
1700123O20RIK
1810006J02RIK
1810013L24RIK
1810022K09RIK
1810030O07RIK
1810032O08RIK
1810037I17RIK
1810041H14RIK
1810043G02RIK
1810055G02RIK
1810058I24RIK
1810062G17RIK
1810062O18RIK
2010107E04RIK
2010111I01RIK
2010300C02RIK
2010315B03RIK
2010320M18RIK
2210016F16RIK
2210016L21RIK
2210039B01RIK
2210408I21RIK
2300009A05RIK
2310001H17RIK
231000

B930036N10RIK
B9D1
B9D2
BABAM1
BABAM2
BACE1
BACH1
BACH2
BAD
BAG1
BAG2
BAG4
BAG5
BAG6
BAHCC1
BAHD1
BAIAP2
BAIAP3
BAMBI
BANF1
BANP
BAP1
BATF
BAX
BAZ1A
BAZ1B
BAZ2A
BAZ2B
BB031773
BBIP1
BBS10
BBS12
BBS2
BBS4
BBS5
BBS7
BBS9
BBX
BC002059
BC003331
BC003965
BC004004
BC005537
BC005561
BC005624
BC017158
BC017643
BC018473
BC028471
BC029722
BC030499
BC031181
BC035044
BC037034
BC043934
BC048403
BC051537
BC052040
BC055324
BC065403
BC147527
BCAP29
BCAP31
BCAS1
BCAS2
BCAS3
BCAT2
BCCIP
BCDIN3D
BCKDHA
BCKDHB
BCKDK
BCL10
BCL11B
BCL2
BCL2L1
BCL2L11
BCL2L12
BCL2L13
BCL2L14
BCL3
BCL6
BCL7A
BCL7B
BCL9
BCL9L
BCLAF1
BCLAF3
BCOR
BCORL1
BCS1L
BDH1
BDKRB1
BDP1
BECN1
BEGAIN
BEND3
BEST1
BET1
BEX2
BEX3
BFAR
BHLHB9
BHLHE40
BICD2
BICDL1
BICRA
BICRAL
BID
BIK
BIN1
BIN2
BIN3
BIRC2
BIRC3
BIRC5
BIRC6
BIVM
BLCAP
BLK
BLM
BLMH
BLNK
BLOC1S1
BLOC1S2
BLOC1S3
BLOC1S4
BLOC1S6
BLVRA
BLVRB
BLZF1
BMF
BMI1
BMP2K
BMPR2
BMS1
BMT2
BNIP1
BNIP2
BNIP3
BNIP3L
BOD1
BOD1L
BOLA1
BOLA2
BOLA3
BOLL
BOP1
BORA
BORCS5
BORCS6
BORCS7
BORCS8
BPGM
BPHL
B

DHX35
DHX36
DHX37
DHX38
DHX40
DHX57
DHX58
DHX8
DHX9
DIABLO
DIAPH1
DIAPH2
DICER1
DIDO1
DIEXF
DIMT1
DIP2B
DIP2C
DIRC2
DIS3
DIS3L
DIS3L2
DISP1
DKC1
DLAT
DLD
DLG1
DLG4
DLGAP4
DLGAP5
DLST
DMAC1
DMAC2
DMAP1
DMTF1
DMXL1
DNA2
DNAAF5
DNAH10
DNAH8
DNAJA1
DNAJA2
DNAJA3
DNAJB1
DNAJB11
DNAJB12
DNAJB13
DNAJB14
DNAJB2
DNAJB4
DNAJB6
DNAJB9
DNAJC1
DNAJC10
DNAJC11
DNAJC12
DNAJC13
DNAJC14
DNAJC15
DNAJC16
DNAJC17
DNAJC18
DNAJC19
DNAJC2
DNAJC21
DNAJC24
DNAJC27
DNAJC3
DNAJC30
DNAJC4
DNAJC5
DNAJC6
DNAJC7
DNAJC8
DNAJC9
DNAL4
DNASE1L1
DNASE1L2
DNASE1L3
DNASE2A
DND1
DNLZ
DNM1L
DNM2
DNMT1
DNMT3A
DNMT3B
DNPEP
DNTT
DNTTIP1
DNTTIP2
DOC2G
DOCK1
DOCK10
DOCK11
DOCK2
DOCK5
DOCK8
DOCK9
DOK1
DOK2
DOLPP1
DONSON
DOPEY1
DOPEY2
DOT1L
DPAGT1
DPF2
DPH1
DPH2
DPH3
DPH5
DPH6
DPH7
DPM2
DPM3
DPP3
DPP4
DPP7
DPP8
DPP9
DPY19L3
DPY19L4
DPY30
DPYSL2
DQX1
DR1
DRAM1
DRAM2
DRAP1
DRC1
DRC3
DRG1
DRG2
DROSHA
DSCC1
DSCR3
DSE
DSN1
DST
DSTN
DSTYK
DTD1
DTD2
DTL
DTNB
DTNBP1
DTWD1
DTWD2
DTX1
DTX2
DTX3
DTX3L
DTYMK
DUBR
DUS1L
DUS2
DUS3L
DUS4L
DUSP1
D

GPRASP1
GPRIN3
GPS1
GPS2
GPSM1
GPSM2
GPSM3
GPT
GPX1
GPX4
GRAMD1A
GRAMD1B
GRAMD3
GRAMD4
GRAP
GRAP2
GRASP
GRB2
GRB7
GRCC10
GREB1
GREB1L
GRHL1
GRHPR
GRIA3
GRIFIN
GRINA
GRIPAP1
GRK2
GRK4
GRK6
GRM6
GRN
GRPEL1
GRPEL2
GRSF1
GRWD1
GSAP
GSDMD
GSE1
GSK3B
GSKIP
GSN
GSPT1
GSR
GSS
GSTA4
GSTCD
GSTK1
GSTM1
GSTM3
GSTM4
GSTO1
GSTP1
GSTP3
GSTT2
GSTZ1
GT(ROSA)26SOR
GTDC1
GTF2A1
GTF2A2
GTF2B
GTF2E1
GTF2E2
GTF2F1
GTF2F2
GTF2H1
GTF2H2
GTF2H3
GTF2H4
GTF2H5
GTF2I
GTF2IRD1
GTF2IRD2
GTF3A
GTF3C1
GTF3C3
GTF3C4
GTF3C5
GTF3C6
GTPBP1
GTPBP10
GTPBP2
GTPBP3
GTPBP4
GTPBP6
GTPBP8
GTSE1
GUCA1B
GUCD1
GUF1
GUK1
GUSB
GXYLT1
GYG
GYPC
GYS1
GZF1
GZMA
GZMB
H13
H1F0
H2-AA
H2-AB1
H2-D1
H2-DMA
H2-DMB2
H2-EB1
H2-K1
H2-KE6
H2-M3
H2-OA
H2-OB
H2-Q1
H2-Q2
H2-Q4
H2-Q6
H2-Q7
H2-T22
H2-T23
H2-T24
H2AFJ
H2AFV
H2AFX
H2AFY
H2AFZ
H3F3A
H3F3B
H6PD
HAAO
HACD1
HACD2
HACD3
HACE1
HACL1
HADH
HADHA
HADHB
HAGH
HAGHL
HARBI1
HARS
HARS2
HAT1
HAUS1
HAUS2
HAUS3
HAUS4
HAUS5
HAUS6
HAUS7
HAUS8
HAX1
HBP1
HBS1L
HCCS
HCFC1
HCFC1R1
HCFC2
HCLS1
HCST
HDAC1
HDAC10

METTL3
METTL4
METTL5
METTL5OS
METTL6
METTL8
METTL9
MEX3A
MEX3B
MEX3C
MFAP1A
MFAP1B
MFAP2
MFAP3
MFF
MFGE8
MFHAS1
MFN1
MFN2
MFNG
MFSD1
MFSD10
MFSD11
MFSD14A
MFSD14B
MFSD3
MFSD4B3
MFSD5
MFSD6
MFSD8
MGA
MGAT1
MGAT4A
MGAT5
MGEA5
MGME1
MGMT
MGRN1
MGST2
MGST3
MIA2
MIA3
MIB1
MIB2
MICAL1
MICAL3
MICU1
MICU2
MICU3
MID1
MIDN
MIEF1
MIEF2
MIEN1
MIER1
MIER2
MIER3
MIF
MIF4GD
MIGA1
MIGA2
MIIP
MILR1
MINDY1
MINDY2
MINDY3
MINK1
MINOS1
MINPP1
MIOS
MIPOL1
MIR155HG
MIR17HG
MIRT1
MIS12
MIS18A
MIS18BP1
MITD1
MITF
MKI67
MKL1
MKL2
MKLN1
MKNK1
MKNK2
MKRN1
MKRN2
MKS1
MLEC
MLF2
MLH1
MLH3
MLLT1
MLLT10
MLLT3
MLLT6
MLST8
MLX
MLXIP
MLYCD
MMAA
MMAB
MMACHC
MMADHC
MMD
MMGT1
MMGT2
MMP11
MMP2
MMP9
MMS19
MMS22L
MNAT1
MNDAL
MNS1
MNT
MOB1A
MOB1B
MOB2
MOB3A
MOB3B
MOB3C
MOB4
MOCS1
MOCS2
MOGS
MON1A
MON1B
MON2
MORC1
MORC2A
MORC3
MORF4L1
MORF4L2
MORN1
MOSPD1
MOSPD2
MOSPD3
MOV10
MPC1
MPC2
MPDU1
MPG
MPHOSPH10
MPHOSPH6
MPHOSPH8
MPHOSPH9
MPI
MPLKIP
MPND
MPP1
MPP5
MPP6
MPP7
MPPE1
MPPED2
MPRIP
MPST
MPV17
MPV17L
MPV17L2
MPZL3
MR1
MRE11A
M

PNPT1
PNRC1
PNRC2
POC1A
POC1B
POC5
PODNL1
PODXL2
POFUT1
POFUT2
POGK
POGLUT1
POGZ
POLA1
POLA2
POLB
POLD1
POLD2
POLD3
POLD4
POLDIP2
POLDIP3
POLE2
POLE3
POLE4
POLG
POLG2
POLH
POLI
POLK
POLL
POLM
POLQ
POLR1A
POLR1B
POLR1C
POLR1D
POLR1E
POLR2A
POLR2B
POLR2C
POLR2D
POLR2E
POLR2F
POLR2G
POLR2H
POLR2I
POLR2J
POLR2K
POLR2L
POLR2M
POLR3A
POLR3B
POLR3C
POLR3D
POLR3E
POLR3F
POLR3G
POLR3GL
POLR3H
POLR3K
POLRMT
POM121
POMGNT1
POMK
POMP
POMT1
POMT2
PON2
PON3
POP1
POP4
POP5
POR
POT1A
POT1B
POU2AF1
POU2F1
POU2F2
POU6F1
PP2D1
PPA1
PPA2
PPAN
PPARD
PPARGC1B
PPAT
PPCDC
PPCS
PPDPF
PPFIA1
PPFIBP1
PPFIBP2
PPHLN1
PPIA
PPIB
PPIC
PPID
PPIE
PPIF
PPIG
PPIH
PPIL1
PPIL2
PPIL3
PPIL4
PPIL6
PPIP5K1
PPIP5K2
PPM1A
PPM1B
PPM1D
PPM1E
PPM1F
PPM1G
PPM1H
PPM1J
PPM1K
PPM1L
PPM1M
PPME1
PPOX
PPP1CA
PPP1CB
PPP1CC
PPP1R10
PPP1R11
PPP1R12A
PPP1R12B
PPP1R12C
PPP1R13B
PPP1R14B
PPP1R15A
PPP1R15B
PPP1R16A
PPP1R16B
PPP1R18
PPP1R1B
PPP1R2
PPP1R21
PPP1R35
PPP1R37
PPP1R3B
PPP1R7
PPP1R8
PPP1R9B
PPP2CA
PPP2CB
PPP2R1A
PPP2R1B
PPP2R2A
PPP2R2D


SHC1
SHCBP1
SHE
SHISA5
SHKBP1
SHMT1
SHMT2
SHOC2
SHPK
SHPRH
SHQ1
SIAE
SIAH1A
SIAH1B
SIAH2
SIAH3
SIDT1
SIDT2
SIGIRR
SIGMAR1
SIK2
SIK3
SIKE1
SIL1
SIMC1
SIN3A
SIN3B
SIPA1
SIPA1L1
SIPA1L2
SIPA1L3
SIRT1
SIRT2
SIRT3
SIRT4
SIRT5
SIRT6
SIRT7
SIT1
SIVA1
SKA2
SKA3
SKAP1
SKAP2
SKI
SKIL
SKIV2L
SKIV2L2
SKP1A
SKP2
SLA
SLA2
SLAIN1
SLAIN2
SLAMF1
SLAMF6
SLAMF7
SLBP
SLC10A7
SLC11A1
SLC11A2
SLC12A2
SLC12A4
SLC12A6
SLC12A7
SLC12A8
SLC12A9
SLC14A1
SLC15A1
SLC15A2
SLC15A4
SLC16A1
SLC16A10
SLC16A13
SLC16A3
SLC16A5
SLC16A6
SLC17A5
SLC17A9
SLC18A2
SLC19A1
SLC19A2
SLC1A5
SLC20A1
SLC22A17
SLC22A23
SLC22A5
SLC23A2
SLC25A1
SLC25A10
SLC25A11
SLC25A12
SLC25A14
SLC25A16
SLC25A17
SLC25A19
SLC25A20
SLC25A22
SLC25A24
SLC25A25
SLC25A26
SLC25A29
SLC25A3
SLC25A30
SLC25A31
SLC25A32
SLC25A35
SLC25A36
SLC25A37
SLC25A38
SLC25A39
SLC25A4
SLC25A40
SLC25A44
SLC25A45
SLC25A46
SLC25A47
SLC25A5
SLC25A51
SLC25A53
SLC26A2
SLC26A6
SLC27A1
SLC27A4
SLC28A2
SLC29A1
SLC29A3
SLC2A1
SLC2A3
SLC2A6
SLC2A8
SLC2A9
SLC30A1
SLC30A4
SLC30A5
SLC30A6


TRAF5
TRAF6
TRAF7
TRAFD1
TRAIP
TRAJ18
TRAJ19
TRAJ2
TRAJ20
TRAJ21
TRAJ50
TRAK1
TRAK2
TRAM1
TRAM2
TRAP1
TRAPPC1
TRAPPC11
TRAPPC12
TRAPPC13
TRAPPC2
TRAPPC2L
TRAPPC3
TRAPPC4
TRAPPC5
TRAPPC6A
TRAPPC6B
TRAPPC8
TRAPPC9
TRAT1
TRAV1
TRAV10
TRAV10N
TRAV12-1
TRAV12-3
TRAV13-1
TRAV13-2
TRAV13N-4
TRAV14-1
TRAV14-2
TRAV14D-3-DV8
TRAV14N-3
TRAV15-1-DV6-1
TRAV16D-DV11
TRAV16N
TRAV17
TRAV19
TRAV2
TRAV21-DV12
TRAV3-1
TRAV3-3
TRAV3-4
TRAV4-2
TRAV4-3
TRAV4-4-DV10
TRAV5-1
TRAV5-4
TRAV6-1
TRAV6-2
TRAV6-3
TRAV6-4
TRAV6D-4
TRAV6D-5
TRAV7-1
TRAV7-3
TRAV7-4
TRAV7-5
TRAV7D-3
TRAV7D-4
TRAV8D-1
TRAV8D-2
TRAV8N-2
TRAV9-1
TRAV9-2
TRAV9-4
TRAV9D-1
TRBC1
TRBC2
TRBJ1-1
TRBJ1-2
TRBJ1-3
TRBJ1-4
TRBJ1-5
TRBJ1-6
TRBJ1-7
TRBJ2-1
TRBJ2-2
TRBJ2-3
TRBJ2-4
TRBJ2-5
TRBJ2-6
TRBJ2-7
TRBV1
TRBV12-1
TRBV12-2
TRBV13-1
TRBV13-2
TRBV13-3
TRBV14
TRBV15
TRBV16
TRBV17
TRBV19
TRBV2
TRBV20
TRBV21
TRBV23
TRBV24
TRBV26
TRBV29
TRBV3
TRBV31
TRBV4
TRBV5
TRDC
TRDMT1
TRDV1
TREML2
TREML4
TRIAP1
TRIB1
TRIB2
TRIM11
TRIM12A
TRIM12C
TRIM13
TRIM14
TRIM1

In [31]:
drode_de_gene_df=pd.read_csv('/home/ivan/Desktop/Project2/MyData/pipeline/DE_gene_fullWT4.csv', sep=',')
full_de_gene = drode_de_gene_df['gene'].unique().tolist()
drode_de_gene = select_de_gene(drode_de_gene_df,score_col='deprob',feature='gene',num=20)

#96 65

98


In [33]:
gene_df_full_CD4sep

,0610009B22RIK,0610009O20RIK,0610010F05RIK,0610012G03RIK,0610030E20RIK,0610037L13RIK,1110002L01RIK,1110004E09RIK,1110004F10RIK,1110008F13RIK,...,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,group,cell
cell,,,,,,,,,,,,,,,,,,,,,
P2275_N716_S505,0.000000,1.005436,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.818314,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,CD69-DP,P2275_N716_S505
P2277_N720_S522,0.000000,0.000000,0.000000,0.000000,0.000000,2.938862,0.000000,0.000000,0.000000,0.000000,...,2.882570,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,CD69+DP,P2277_N720_S522
P2280_N714_S517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,TCR+DP,P2280_N714_S517
P2275_N719_S510,0.000000,0.000000,0.000000,0.000000,2.400052,0.000000,0.000000,0.000000,0.507921,0.000000,...,0.000000,0.000000,0.0,2.476722,0.000000,0.000000,0.000000,0.000000,CD69+DP,P2275_N719_S510
P2222_N710_S508,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.453933,0.000000,CD8SP,P2222_N710_S508
P2222_N707_S504,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.402210,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,CD4SP_mature,P2222_N707_S504
P2279_N723_S502,0.000000,0.000000,0.000000,0.000000,0.000000,1.571551,0.000000,0.000000,1.628096,0.013248,...,0.000000,2.849742,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,CD4SP_mature,P2279_N723_S502
P2276_N705_S521,0.000000,0.000000,0.000000,0.000000,0.079973,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.744859,0.000000,0.000000,0.000000,CD4+CD8low,P2276_N705_S521
P2222_N705_S505,0.000000,0.000000,0.000000,0.000000,0.000000,1.460457,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,CD4+CD8low,P2222_N705_S505


In [32]:
gene_dfc=gene_df_full.copy()
gene_dfc['cell']=gene_dfc.index
gene_dfc=gene_dfc.reset_index(drop=True)
#Part 3 generate clustered WT4 subgroups(WT4_sub) as 'small' and 'large'

lst = cluster_cells(gene_dfc,drode_de_gene,group='CD4SP',group_col='group',id_col='cell')
small_lst=lst[0:22]+[lst[len(lst)-1]]
large_lst=lst[22:len(lst)-1]
gene_dfc.index=gene_dfc['cell']
gene_dfc['WT4_sub']= gene_dfc['group']
gene_dfc.loc[small_lst,'WT4_sub']='CD4SP_immature'
gene_dfc.loc[large_lst,'WT4_sub']='CD4SP_mature'
gene_dfc['group']=gene_dfc['WT4_sub']
del gene_dfc['WT4_sub']
gene_df_full_CD4sep=gene_dfc.copy()
gene_dfc=gene_dfc[(gene_dfc['group']!='CD4SP_immature') & (gene_dfc['group']!='TCR+DP')]
print(gene_dfc['group'].unique())



['CD69-DP' 'CD69+DP' 'CD8SP' 'CD4SP_mature' 'CD4+CD8low']


In [26]:
gene_df_full_CD4sep[gene_df_full_CD4sep['group']=='CD4SP_mature'][['ITM2A','CD24A','CD69','CCR9','STAT1','CCR7','H2-Q7']].describe()

,ITM2A,CD24A,CD69,CCR9,STAT1,CCR7,H2-Q7
count,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000
mean,1.256416,0.077882,0.474429,0.127930,2.298884,2.115175,1.332501
std,1.508925,0.454231,1.075106,0.600613,1.654969,1.554307,1.272986
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.121379,0.000000,0.000000
50%,0.014235,0.000000,0.000000,0.000000,2.883503,2.830304,1.521403
75%,2.788892,0.000000,0.015241,0.000000,3.528061,3.438451,2.438685
max,4.227945,2.755320,4.173711,3.371386,4.960322,4.210364,3.512027


In [27]:
gene_df_full_CD4sep[gene_df_full_CD4sep['group']=='CD4SP_immature'][['ITM2A','CD24A','CD69','CCR9','STAT1','CCR7','H2-Q7']].describe()

,ITM2A,CD24A,CD69,CCR9,STAT1,CCR7,H2-Q7
count,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,3.294199,0.113620,1.120446,0.846407,0.832117,1.542397,0.089607
std,1.923285,0.544902,1.483695,1.271555,1.455250,1.741940,0.429738
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.756851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.793403,0.000000,0.000000,0.000000,0.000000,0.040962,0.000000
75%,4.433352,0.000000,2.472780,2.050417,1.278382,3.340918,0.000000
max,5.514202,2.613257,4.078529,3.369131,4.994376,4.131553,2.060953


In [10]:
drode_de_gene_df = Drode_DE_gene_detection(gene_dfc,full_de_gene,feature='gene',id_col='cell',group_col='group',is_unify=True)
drode_de_gene_df.to_csv(
    path_or_buf="/home/ivan/Desktop/Project2/MyData/pipeline/DE_gene_matureWT4.csv", 
    sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, 
    mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', 
    chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, escapechar=None, decimal='.')

0610009B22RIK
0610009O20RIK
0610010F05RIK
0610012G03RIK
0610030E20RIK
0610037L13RIK
1110002L01RIK
1110004E09RIK
1110004F10RIK
1110008F13RIK
1110008P14RIK
1110012L19RIK
1110032A03RIK
1110034G24RIK
1110038F14RIK
1110051M20RIK
1110059E24RIK
1110059G10RIK
1190002N15RIK
1190007I07RIK
1300002E11RIK
1500011B03RIK
1500011K16RIK
1500015A07RIK
1600002K03RIK
1600014C10RIK
1600020E01RIK
1700017B05RIK
1700020I14RIK
1700021F05RIK
1700030K09RIK
1700037H04RIK
1700062C10RIK
1700066M21RIK
1700096K18RIK
1700097N02RIK
1700123O20RIK
1810013L24RIK
1810022K09RIK
1810030O07RIK
1810032O08RIK
1810037I17RIK
1810041H14RIK
1810043G02RIK
1810055G02RIK
1810058I24RIK
1810062O18RIK
2010107E04RIK
2010111I01RIK
2010300C02RIK
2010315B03RIK
2010320M18RIK
2210016F16RIK
2210016L21RIK
2210039B01RIK
2300009A05RIK
2310001H17RIK
2310009B15RIK
2310011J03RIK
2310022A10RIK
2310033P09RIK
2310035C23RIK
2310057M21RIK
2310061I04RIK
2410002F23RIK
2410004B18RIK
2410015M20RIK
2410018L13RIK
2410089E03RIK
2410131K14RIK
2510002D24RIK
251003

CALM2
CALM3
CALR
CALU
CAMK1D
CAMK2D
CAMK2G
CAMK4
CAMKK1
CAMKK2
CAMKMT
CAML
CAMSAP1
CAMTA1
CAMTA2
CAND1
CAND2
CANT1
CANX
CAP1
CAPG
CAPN1
CAPN10
CAPN2
CAPN3
CAPN7
CAPNS1
CAPRIN1
CAPZA1
CAPZA2
CAPZB
CAR2
CAR9
CARD11
CARD19
CARD6
CARD9
CARF
CARHSP1
CARM1
CARMIL2
CARNMT1
CARNS1
CARS
CARS2
CASC3
CASC4
CASD1
CASP2
CASP3
CASP6
CASP7
CASP8
CASP8AP2
CASS4
CAST
CAT
CBFA2T2
CBFA2T3
CBFB
CBL
CBLB
CBLL1
CBR1
CBR4
CBWD1
CBX1
CBX2
CBX3
CBX4
CBX5
CBX7
CBX8
CBY1
CC2D1A
CC2D1B
CCAR1
CCAR2
CCDC102A
CCDC107
CCDC115
CCDC117
CCDC12
CCDC124
CCDC125
CCDC127
CCDC130
CCDC134
CCDC137
CCDC138
CCDC14
CCDC153
CCDC163
CCDC166
CCDC17
CCDC171
CCDC173
CCDC174
CCDC181
CCDC186
CCDC191
CCDC22
CCDC25
CCDC28A
CCDC28B
CCDC30
CCDC32
CCDC43
CCDC47
CCDC50
CCDC51
CCDC58
CCDC59
CCDC6
CCDC61
CCDC62
CCDC66
CCDC69
CCDC71
CCDC77
CCDC82
CCDC84
CCDC85B
CCDC86
CCDC88B
CCDC88C
CCDC9
CCDC90B
CCDC91
CCDC92
CCDC93
CCDC94
CCDC97
CCHCR1
CCL25
CCL4
CCM2
CCNB2
CCNC
CCND2
CCND3
CCNDBP1
CCNE1
CCNG1
CCNG2
CCNH
CCNI
CCNJ
CCNK
CCNL1
CCNL2
CCNT1
CCNT2

EXOC3
EXOC4
EXOC5
EXOC6
EXOC6B
EXOC7
EXOC8
EXOG
EXOSC1
EXOSC10
EXOSC2
EXOSC3
EXOSC4
EXOSC5
EXOSC7
EXOSC8
EXOSC9
EXT1
EXT2
EXTL3
EYA2
EYA3
EZH1
EZH2
EZR
F13A1
F2R
F730311O21RIK
FAAH
FAAP100
FAAP20
FAAP24
FABP5
FADD
FADS1
FAF1
FAF2
FAHD2A
FAIM
FAM102A
FAM102B
FAM103A1
FAM104A
FAM105A
FAM107B
FAM110A
FAM111A
FAM114A2
FAM117A
FAM118A
FAM118B
FAM120A
FAM120AOS
FAM120B
FAM120C
FAM126A
FAM126B
FAM129C
FAM133B
FAM136A
FAM13B
FAM149B
FAM160A2
FAM160B1
FAM160B2
FAM161B
FAM162A
FAM168A
FAM168B
FAM169B
FAM172A
FAM173A
FAM173B
FAM174A
FAM186B
FAM189B
FAM192A
FAM193A
FAM193B
FAM204A
FAM206A
FAM207A
FAM208A
FAM208B
FAM20B
FAM210A
FAM213A
FAM214A
FAM216A
FAM219B
FAM220A
FAM221A
FAM222B
FAM234A
FAM234B
FAM241A
FAM26F
FAM32A
FAM35A
FAM3A
FAM3C
FAM45A
FAM49A
FAM49B
FAM50A
FAM53A
FAM53B
FAM53C
FAM58B
FAM60A
FAM69A
FAM76A
FAM76B
FAM78A
FAM84B
FAM8A1
FAM91A1
FAM92A
FAM96A
FAM96B
FAM98A
FAM98B
FAM98C
FANCC
FANCE
FANCG
FANCL
FANCM
FAR1
FARP1
FARS2
FARSA
FARSB
FAS
FASN
FASTK
FASTKD1
FASTKD2
FASTKD3
FAU
FBF1
FB

KLC4
KLF10
KLF13
KLF2
KLF3
KLF6
KLF7
KLHDC1
KLHDC2
KLHDC3
KLHDC4
KLHDC8B
KLHL12
KLHL18
KLHL20
KLHL21
KLHL22
KLHL24
KLHL25
KLHL26
KLHL28
KLHL3
KLHL36
KLHL42
KLHL6
KLHL7
KLHL9
KLK8
KLRD1
KMT2A
KMT2B
KMT2C
KMT2D
KMT2E
KMT5A
KMT5B
KMT5C
KNOP1
KNSTRN
KPNA1
KPNA2
KPNA3
KPNA4
KPNA6
KPNB1
KPTN
KRAS
KRBA1
KRCC1
KREMEN1
KREMEN2
KRI1
KRIT1
KRR1
KRT10
KRTCAP2
KRTCAP3
KTN1
KXD1
KYAT1
KYAT3
L3MBTL2
L3MBTL3
LACTB
LACTB2
LAGE3
LAMB3
LAMP1
LAMP2
LAMTOR1
LAMTOR2
LAMTOR3
LAMTOR4
LAMTOR5
LANCL1
LANCL2
LAP3
LAPTM4A
LAPTM4B
LAPTM5
LARP1
LARP4
LARP4B
LARP7
LARS
LARS2
LAS1L
LASP1
LAT
LATS1
LAX1
LBH
LBR
LCA5
LCK
LCLAT1
LCMT1
LCMT2
LCN4
LCOR
LCORL
LCP1
LCP2
LDAH
LDB1
LDHA
LDHB
LDHD
LDLR
LDLRAD3
LDLRAD4
LDLRAP1
LEF1
LEMD2
LENG1
LENG8
LENG9
LEO1
LEPROT
LEPROTL1
LETM1
LETM2
LETMD1
LFNG
LGALS1
LGALS3
LGALS3BP
LGALS8
LGALS9
LGMN
LIAS
LIG1
LIG3
LIG4
LIMD1
LIMD2
LIMK1
LIMK2
LIMS1
LIN37
LIN52
LIN7C
LIN9
LINS1
LIPA
LIPE
LIPO3
LITAF
LLGL1
LLGL2
LLPH
LMAN1
LMAN2
LMAN2L
LMBR1L
LMBRD1
LMBRD2
LMF1
LMF2
LMNA
LMNB1
LMNB2
LMO4


PAXIP1
PAXX
PBDC1
PBRM1
PBX4
PBXIP1
PCBD2
PCBP1
PCBP2
PCBP4
PCCA
PCED1A
PCED1B
PCF11
PCGF1
PCGF3
PCGF5
PCGF6
PCID2
PCIF1
PCM1
PCMT1
PCMTD1
PCMTD2
PCNA
PCNP
PCNT
PCNX
PCNX3
PCNX4
PCSK7
PCYT1A
PCYT2
PDAP1
PDCD1
PDCD10
PDCD11
PDCD1LG2
PDCD2
PDCD2L
PDCD4
PDCD5
PDCD6
PDCD6IP
PDCD7
PDCL
PDCL3
PDE12
PDE1B
PDE3B
PDE4B
PDE4D
PDE4DIP
PDE6D
PDE7A
PDHA1
PDHB
PDHX
PDIA3
PDIA4
PDIA6
PDIK1L
PDK1
PDK2
PDK3
PDLIM1
PDLIM2
PDLIM4
PDLIM5
PDLIM7
PDP1
PDP2
PDPK1
PDPR
PDRG1
PDS5A
PDS5B
PDSS1
PDSS2
PDXDC1
PDXK
PDXP
PDZD11
PDZD8
PEA15A
PEAK1
PEBP1
PECAM1
PEF1
PELI1
PELP1
PEPD
PER1
PES1
PET100
PEX1
PEX10
PEX11G
PEX12
PEX13
PEX14
PEX16
PEX19
PEX2
PEX3
PEX5
PEX6
PEX7
PFAS
PFDN1
PFDN2
PFDN4
PFDN5
PFDN6
PFKL
PFKM
PFKP
PFN1
PGAM1
PGAM5
PGAP2
PGD
PGGHG
PGGT1B
PGK1
PGLS
PGLYRP2
PGM1
PGM2
PGM2L1
PGM3
PGP
PGPEP1
PGRMC1
PGRMC2
PGS1
PHACTR2
PHAX
PHB
PHB2
PHC2
PHC3
PHF1
PHF11A
PHF11B
PHF11C
PHF12
PHF14
PHF2
PHF20
PHF20L1
PHF21A
PHF23
PHF3
PHF5A
PHF6
PHF7
PHF8
PHGDH
PHIP
PHKB
PHKG2
PHLDA1
PHOSPHO2
PHRF1
PHTF1
PHTF2
PHYH
PHY

SEC31A
SEC61A1
SEC61A2
SEC61B
SEC61G
SEC62
SEC63
SECISBP2
SECISBP2L
SEH1L
SEL1L
SELENOF
SELENOH
SELENOK
SELENOO
SELENOP
SELENOS
SELENOT
SELENOW
SELL
SELPLG
SEM1
SEMA4A
SEMA4B
SEMA4D
SENP1
SENP2
SENP3
SENP5
SENP6
SENP7
SEPHS1
SEPHS2
SEPSECS
SEPT1
SEPT11
SEPT6
SEPT7
SEPT9
SERBP1
SERF2
SERGEF
SERHL
SERINC1
SERINC3
SERINC5
SERP1
SERP2
SERPINB6A
SERPINB6B
SERPINB9
SERPINI1
SERTAD1
SERTAD2
SERTAD3
SESN1
SESN3
SESTD1
SET
SETD2
SETD3
SETD4
SETD5
SETD6
SETD7
SETDB1
SETDB2
SETX
SF1
SF3A1
SF3A3
SF3B1
SF3B2
SF3B3
SF3B4
SF3B5
SF3B6
SFI1
SFMBT1
SFN
SFPQ
SFR1
SFSWAP
SFT2D1
SFT2D2
SFT2D3
SFXN1
SFXN2
SFXN3
SFXN4
SGCB
SGF29
SGK1
SGK3
SGMS1
SGO2A
SGPL1
SGPP1
SGTA
SH2B1
SH2B3
SH2D1A
SH2D2A
SH2D3C
SH3BGRL
SH3BGRL3
SH3BP1
SH3BP2
SH3BP5
SH3BP5L
SH3GL1
SH3GLB1
SH3GLB2
SH3KBP1
SH3PXD2A
SH3RF1
SHARPIN
SHC1
SHISA5
SHKBP1
SHMT1
SHMT2
SHOC2
SHPRH
SIAE
SIAH1A
SIDT1
SIDT2
SIGIRR
SIGMAR1
SIK2
SIK3
SIKE1
SIL1
SIN3A
SIN3B
SIPA1
SIPA1L1
SIPA1L2
SIRT2
SIRT3
SIRT4
SIRT5
SIRT6
SIRT7
SIT1
SIVA1
SKA2
SKA3
SKAP1
SKAP2
SKI
SKI

TRAV3-4
TRAV4-2
TRAV4-3
TRAV4-4-DV10
TRAV5-1
TRAV5-4
TRAV6-2
TRAV6-3
TRAV6-4
TRAV6D-4
TRAV6D-5
TRAV7-1
TRAV7-3
TRAV7-4
TRAV7-5
TRAV7D-3
TRAV7D-4
TRAV8D-1
TRAV8D-2
TRAV9-1
TRAV9-2
TRAV9-4
TRAV9D-1
TRBC1
TRBC2
TRBJ1-1
TRBJ1-2
TRBJ1-3
TRBJ1-4
TRBJ1-5
TRBJ1-6
TRBJ1-7
TRBJ2-1
TRBJ2-2
TRBJ2-3
TRBJ2-4
TRBJ2-5
TRBJ2-6
TRBJ2-7
TRBV1
TRBV12-1
TRBV12-2
TRBV13-1
TRBV13-2
TRBV13-3
TRBV14
TRBV15
TRBV16
TRBV17
TRBV19
TRBV2
TRBV20
TRBV24
TRBV26
TRBV29
TRBV3
TRBV31
TRBV4
TRBV5
TRDC
TRDMT1
TREML2
TRIAP1
TRIB1
TRIB2
TRIM11
TRIM12A
TRIM12C
TRIM13
TRIM14
TRIM17
TRIM21
TRIM23
TRIM24
TRIM25
TRIM26
TRIM27
TRIM28
TRIM30A
TRIM30D
TRIM33
TRIM34A
TRIM34B
TRIM35
TRIM36
TRIM37
TRIM39
TRIM41
TRIM44
TRIM5
TRIM56
TRIM59
TRIM65
TRIM8
TRIOBP
TRIP11
TRIP12
TRIP4
TRIR
TRIT1
TRMO
TRMT1
TRMT10A
TRMT10B
TRMT10C
TRMT11
TRMT112
TRMT12
TRMT13
TRMT1L
TRMT2A
TRMT2B
TRMT44
TRMT5
TRMT6
TRMU
TRNAU1AP
TRNT1
TRP53
TRP53BP1
TRP53BP2
TRP53I11
TRP53I13
TRP53INP1
TRP53INP2
TRP53RKA
TRP53RKB
TRPC4AP
TRPM1
TRPM7
TRPS1
TRPV2
TRRAP
TRUB1
TRUB

In [42]:
drode_de_gene_df=pd.read_csv('/home/ivan/Desktop/Project2/MyData/pipeline/DE_gene_matureWT4.csv', sep=',')
full_de_gene = drode_de_gene_df['gene'].unique().tolist()
drode_de_gene_df=drode_de_gene_df[(drode_de_gene_df['group_1']!='TCR+DP') & (drode_de_gene_df['group_2']!='TCR+DP')].copy()
drode_de_gene = select_de_gene(drode_de_gene_df,score_col='deprob',feature='gene',num=21)
pd.DataFrame(drode_de_gene).to_csv(
    path_or_buf="/home/ivan/Desktop/Project2/MyData/pipeline/degenes_drode.csv", 
    sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, 
    mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', 
    chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, escapechar=None, decimal='.')

100


In [12]:
#line charts for de genes
gene_expression_level_df,gene_expression_rate_df = de_gene_description(drode_de_gene_df)
gene_expression_level_df = gene_expression_level_df.loc[['CD69-DP','CD69+DP','CD4+CD8low','CD4SP_mature','CD8SP']]
gene_expression_rate_df = gene_expression_rate_df.loc[['CD69-DP','CD69+DP','CD4+CD8low','CD4SP_mature','CD8SP']]
plot_line_charts(gene_expression_level_df[['CD4_expression_level','CD8A_expression_level']],df2=gene_expression_rate_df[['CD4_expression_rate','CD8A_expression_rate']],width=1400,height=900,titleName='Gene Expression Rates and Levels',xlabel='subpopulations',ylabel='expression_level',ylabel_sec='expression_rate',outputFilePath = "/home/ivan/Desktop/Project2/MyData/pipeline/rate_and_level.html")


In [29]:
#pca plots
principalDf,pca_description= pca_transformation(gene_df_full_CD4sep,drode_de_gene)
for i in range(1,3):
    for j in range(i+1,4):
        plotColorScatter_x(principalDf ,xvalue = 'pc'+str(i),yvalue = 'pc'+str(j), sizevalue = 10, \
                         outputFilePath='/home/ivan/Desktop/Project2/MyData/pipeline/'+str(len(drode_de_gene))+'pca'+str(i)+'-'+str(j)+'.html',\
                         plotWidth = 1000, plotHeight =1000, readList = ['cell','group','pc'+str(i),'pc'+str(j)],\
                         titleName='pc'+str(i)+' vs pc'+str(j), colorColumn="group")

In [14]:
#plot the VLM plot for genes

principalDf.loc[principalDf['group']=='CD69-DP','newPlotGroup']=1-np.random.random(principalDf[principalDf['group']=='CD69-DP'].shape[0])*0.8
principalDf.loc[principalDf['group']=='CD69+DP','newPlotGroup']=2-np.random.random(principalDf[principalDf['group']=='CD69+DP'].shape[0])*0.8
principalDf.loc[principalDf['group']=='CD4+CD8low','newPlotGroup']=3-np.random.random(principalDf[principalDf['group']=='CD4+CD8low'].shape[0])*0.8
principalDf.loc[principalDf['group']=='CD4SP_immature','newPlotGroup']=4-np.random.random(principalDf[principalDf['group']=='CD4SP_immature'].shape[0])*0.8
principalDf.loc[principalDf['group']=='CD4SP_mature','newPlotGroup']=5-np.random.random(principalDf[principalDf['group']=='CD4SP_mature'].shape[0])*0.8
principalDf.loc[principalDf['group']=='CD8SP','newPlotGroup']=6-np.random.random(principalDf[principalDf['group']=='CD8SP'].shape[0])*0.8
principalDf.loc[principalDf['group']=='TCR+DP','newPlotGroup']=7-np.random.random(principalDf[principalDf['group']=='TCR+DP'].shape[0])*0.8
principalDf=principalDf.reset_index(drop=True)
for i in ['pc1','pc2','pc3']:
    plotColorScatter_x(principalDf ,xvalue = 'newPlotGroup',yvalue = i, sizevalue = 5, outputFilePath='/home/ivan/Desktop/Project2/MyData/pipeline/'+i+'.html',plotWidth = 1500, plotHeight = 1000, readList = ['group'],titleName=i, colorColumn="group", colorPattern=viridis)


In [15]:
h_bar_plot(pca_description.sort_values(by='rank-pc1')[0:20].copy(),'gene','pc1','rank-pc1',bottom_up=False,height=600,title='Gene Contribution for PC1',file_path='/home/ivan/Desktop/Project2/MyData/pipeline/pc1_genes.html')
h_bar_plot(pca_description.sort_values(by='rank-pc2')[0:20].copy(),'gene','pc2','rank-pc2',bottom_up=False,height=600,title='Gene Contribution for PC2',file_path='/home/ivan/Desktop/Project2/MyData/pipeline/pc2_genes.html')
h_bar_plot(pca_description.sort_values(by='rank-pc3')[0:20].copy(),'gene','pc3','rank-pc3',bottom_up=False,height=600,title='Gene Contribution for PC3',file_path='/home/ivan/Desktop/Project2/MyData/pipeline/pc3_genes.html')

In [40]:
both_de_gene = list(set(seurat_degenes+drode_de_gene))
principalDf,pca_description= pca_transformation(gene_df_full_CD4sep,both_de_gene)
for i in range(1,3):
    for j in range(i+1,4):
        plotColorScatter_x(principalDf ,xvalue = 'pc'+str(i),yvalue = 'pc'+str(j), sizevalue = 10, \
                         outputFilePath='/home/ivan/Desktop/Project2/MyData/pipeline/'+str(len(both_de_gene))+'pca'+str(i)+'-'+str(j)+'.html',\
                         plotWidth = 1000, plotHeight =1000, readList = ['cell','group','pc'+str(i),'pc'+str(j)],\
                         titleName='pc'+str(i)+' vs pc'+str(j), colorColumn="group")

In [20]:
rna_df = gene_df_full_CD4sep[['cell','group']+drode_de_gene].copy()
rna_df=rna_df.reset_index(drop=True)
rna_scaled_df,select_major_mean = unify_df(rna_df,drode_de_gene,'cell',['group'])
rna_df=rna_scaled_df.copy()
select_major=select_major_mean.copy()
select_major.remove('CD4')
select_major.remove('CD8A')
select_major.remove('CD8B1')
WT1245_df = rna_df[(rna_df['group']!='CD4SP_immature') & (rna_df['group']!='TCR+DP')].copy()
WT2_df = rna_df[(rna_df['group']=='CD4SP_immature') | (rna_df['group']=='TCR+DP')].copy()
training_X2,testing_X2,training_Y2,testing_Y2 = cross_validation_split_with_unbalance_data(WT2_df,select_major,label='group',id_column='cell',test_size=0.2,handle_unbalance=False)
training_X2['group']=training_Y2
testing_X2['group']=testing_Y2
for i in range(0,60,1):
    training_X1,testing_X1,training_Y1,testing_Y1 = cross_validation_split_with_unbalance_data(WT1245_df,select_major,label='group',id_column='cell',test_size=0.2,handle_unbalance=False)
    training_X1['group']=training_Y1
    testing_X1['group']=testing_Y1
    testing_X1 = testing_X1.append(training_X2)
    testing_X1 = testing_X1.append(testing_X2)
    df_result = combined_eXGBT_classifier(training_set = training_X1,numeric_features_validation = select_major,testing_set = testing_X1,label_column = 'group',max_depth=2,num_trees=50)
    full_result,clean_result = transform_predict_result_DF(predict_result_DF =df_result, label_col='group', threshold=0.1)
    full_result[['mean_from_unify_df','group','CD69-DP','CD69+DP','CD4+CD8low','CD4SP_mature','CD8SP','multi_eXGBT_pre_label', 'predict_result_DF_indices','F_label']].to_csv(
    path_or_buf="/home/ivan/Desktop/Project2/MyData/pipeline/xgboost/"+str(i)+".csv", 
    sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, 
    mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', 
    chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, escapechar=None, decimal='.')

Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: The predicted labels were used instead of full labels
Notice: 

In [22]:

xgb_path = "/home/ivan/Desktop/Project2/MyData/pipeline/xgboost/"
files = os.listdir(xgb_path) 
files = [xgb_path+f for f in files if f.endswith('.csv') if not f.startswith('.')] 
xgb_df = pd.DataFrame()
for file in files:
    temp_df = pd.read_csv(file, sep=',')
    xgb_df=xgb_df.append(temp_df,sort=False)
xgb_df['cell']=xgb_df.index
score_df= xgb_df[['cell','mean_from_unify_df','group','CD69-DP','CD4SP_mature','CD8SP','multi_eXGBT_pre_label', 'predict_result_DF_indices','F_label']].copy()
score_df['size']=score_df['CD69-DP']*15+3
assert len(score_df['predict_result_DF_indices'].unique())==len(rna_df['cell'].unique())
score_df.to_csv(
    path_or_buf=xgb_path+"full_result.csv", 
    sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, 
    mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', 
    chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, escapechar=None, decimal='.')
temp_df = score_df[(score_df['group']=='CD69-DP') | (score_df['group']=='CD4SP_mature')| (score_df['group']=='CD8SP')].copy()
temp_df['correct']=temp_df['group']==temp_df['F_label']
print('precision: '+str(temp_df[temp_df['correct']==True].shape[0]/(temp_df[temp_df['correct']==False].shape[0]+temp_df[temp_df['correct']==True].shape[0])))
plot_df_i = score_df.groupby(by=['group','predict_result_DF_indices'],as_index=False).min()
plot_df_i['CD4SP_mature']=plot_df_i['CD4SP_mature']-plot_df_i['CD69-DP']+1
plot_df_i['CD8SP']=plot_df_i['CD8SP']-plot_df_i['CD69-DP']+1
plot_df_i['CD8SP']=plot_df_i['CD8SP']/plot_df_i['CD8SP'].max()
plot_df_i['CD4SP_mature']=plot_df_i['CD4SP_mature']/plot_df_i['CD4SP_mature'].max()
plotColorScatter_x(plot_df_i ,xvalue = 'CD4SP_mature',yvalue = 'CD8SP', sizevalue =10, outputFilePath='/home/ivan/Desktop/Project2/MyData/pipeline/plot_df_avg.html',plotWidth = 1000, plotHeight = 1000, readList = ['group','cell','CD4SP_mature','CD8SP'],titleName='trajectories of T cell differentiation', colorColumn="group")


precision: 0.9198803191489362
